### Import Libraries

In [3]:
import pandas as pd
import numpy as np 

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Users\khair\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

### LOAD DATASET

In [ ]:
df = pd.read_csv("dataset/movies_metadata.csv", low_memory=False)

print("Data awal:", df.shape)
df.head()


Data awal: (45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### PREPOCESSING

In [ ]:
# Ambil kolom penting
df = df[['title', 'overview']]

# Hapus data kosong
df.dropna(inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)

# Normalisasi judul untuk input user
df['title_clean'] = df['title'].str.lower().str.strip()

print("Setelah preprocessing:", df.shape)
df.head()


Setelah preprocessing: (44506, 3)


,title,overview,title_clean
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",toy story
1,Jumanji,When siblings Judy and Peter discover an encha...,jumanji
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,grumpier old men
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",waiting to exhale
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,father of the bride part ii


### LOAD MODEL TRANSFORMER

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


### EMBEDDING SINOPSIS

In [ ]:
movie_embeddings = model.encode(
    df['overview'].tolist(),
    show_progress_bar=True
)


Batches: 100%|██████████| 1391/1391 [08:05<00:00,  2.87it/s]


### FUNGSI CARI JUDUL CERDAS

In [ ]:
def recommend_movie(query, top_n=5):
    query = query.lower().strip()

    # Cari judul yang mengandung input
    candidates = df[df['title_clean'].str.contains(query)]

    if len(candidates) == 0:
        return "Judul film tidak ditemukan dalam dataset."

    # Ambil film pertama yang cocok
    idx = candidates.index[0]

    query_embedding = movie_embeddings[idx].reshape(1, -1)

    similarities = cosine_similarity(query_embedding, movie_embeddings)[0]

    similar_indices = similarities.argsort()[::-1][1:top_n+1]

    result = df.iloc[similar_indices][['title']].copy()
    result['Skor Kemiripan'] = similarities[similar_indices]

    return result.reset_index(drop=True)


### UJI COBA SISTEM

In [ ]:
recommend_movie("Batman", top_n=5)


,title,Skor Kemiripan
0,The Dark Knight Rises,0.681206
1,Batman: Year One,0.626658
2,Batman,0.609171
3,Batman Beyond: Return of the Joker,0.589893
4,Batman & Robin,0.584853


### MODE INTERAKTIF USER

In [ ]:
while True:
    judul = input("\nMasukkan judul film (ketik 'exit' untuk keluar): ")
    
    if judul.lower() == 'exit':
        break
        
    hasil = recommend_movie(judul, top_n=5)
    print(hasil)


                                title  Skor Kemiripan
0               The Dark Knight Rises        0.681206
1                    Batman: Year One        0.626658
2                              Batman        0.609171
3  Batman Beyond: Return of the Joker        0.589893
4                      Batman & Robin        0.584853
